구글 드라이브에 있는 csv 파일을 import하여 코랩에서 실행한다 가정하고 작성하였습니다.

In [103]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1-1. Import Data

In [104]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F #torch.nn.functional.logsigmoid 함수 사용

In [105]:
import csv
import pandas as pd

#아래 주소는 적절하게 바꾸면 됩니다.
data_csv = '/content/drive/MyDrive/data.csv'

data = pd.read_csv(data_csv, header=0, encoding='utf-8')

## 1-2. Data preprocessing

In [106]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import math

BATCH_SIZE = 10 #실험을 통해 수정

kind = {'한식':0, '양식':1, '아시아음식':2, '일식':3, '중식':4, '분식':5, '카페':6, '뷔페':7, '기타':8}

#가게이름 | 음식종류 | 가격 | 구글 지도 별점 | 방문자 리뷰 | 블로그 리뷰 | 주소 | 주관적 별점
class CustomDataset(Dataset):
  def __init__(self, dataframe):
    self.data = dataframe

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    _names = self.data.iloc[idx, 0]
    _address = self.data.iloc[idx, 1]
    _kinds = [kind[self.data.iloc[idx, 2]] for i in range(9)]
    _x = self.data.iloc[idx, 3:7].astype(np.float32)
    _target_score = self.data.iloc[idx, 7].astype(np.float32)

    return torch.tensor(_kinds), torch.tensor(_x), torch.tensor(_target_score)

X_train, X_test = train_test_split(data, test_size=0.33, random_state=1230)

training_dataset = CustomDataset(X_train)
training_loader = DataLoader(training_dataset, batch_size = BATCH_SIZE, shuffle = True)
test_dataset = CustomDataset(X_test)
test_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle = True)

In [107]:
X_train

,가게 이름,음식 종류,주소,가격,네이버 지도 별점,방문자 리뷰?,블로그 리뷰,주관적 별점?
261,육장장이,경기 수원시 영통구 청명남로 32 월드프라자 2층 육장장이,한식,19900,4.33,190,158,4.5
213,동백카츠,대전 서구 둔산로31번길 56,양식,13900,4.50,1090,853,4.0
276,유쾌한회세꼬시,경기 수원시 영통구 청명남로50번길 5-8,일식,40000,4.60,106,46,3.0
177,도스마스,경기 용인시 기흥구 서그내로15번길 33,양식,5000,4.28,264,24,3.5
111,정월,서울 강남구 강남대로102길 46,카페,7000,4.46,2012,1533,4.0
...,...,...,...,...,...,...,...,...
1,홍미닭발,서울 송파구 백제고분로7길 24-13,아시아음식,23000,4.43,248,207,4.0
41,미소야 대전신성점,대전 유성구 신성로72번길 64,일식,11000,4.32,55,11,3.0
225,역전할머니맥주 영통점,경기 수원시 영통구 반달로35번길 30 1층 역전할머니맥주,한식,4500,4.33,341,55,3.0
253,매스버거,경기 수원시 영통구 청명남로4번길 8 1층 102호,양식,8500,4.91,496,132,4.0


## 2. Define Mapping function

In [108]:
def Mapping(score):
  scaled_score= F.softmax(score)

  return scaled_score[:, 0].unsqueeze(1)

## 3. MLP nodel define

In [109]:
class MLP(nn.Module):
  def __init__(self, node_number, device):
    super(MLP, self).__init__()
    self.MLP = nn.Sequential(
        nn.Linear(5, node_number),
        nn.Linear(node_number, node_number*2),
        nn.Linear(node_number*2, 2)
    )
    #음식 종류 기준 : 네이버
    #한식, 양식, 아시아음식, 일식, 중식, 분식, 카페, 뷔페, 기타
    self.weight_0 = nn.Parameter(torch.full((9,), 0.5, dtype=torch.float32))

  def forward(self, x):
    score = self.MLP(x)

    return score

## 6. TRAINING



> MLP Training



In [110]:
def compute_accuracy_and_loss(model, data_loader, device):
    cross_entropy = 0
    num_examples = 0

    for batch_idx, (_kinds, _x, _target_scores) in enumerate(data_loader):
      kinds = torch.tensor([torch.matmul(_kinds[i].to(torch.float32), model.weight_0.to(torch.float32)) for i in range(len(_kinds))])
      x = torch.cat((kinds.unsqueeze(1), _x), dim=1)
      x = model(x.to(DEVICE))

      target_scores = _target_scores/5

      cross_entropy += loss(Mapping(x), target_scores.unsqueeze(1))
      num_examples += 1

    return cross_entropy/num_examples

In [111]:
import torch.nn.init as init

def init_weights(model):
  for name, param in model.named_parameters():
    if 'weight' in name:
      init.normal_(param, mean=0, std=0.01)
    elif 'bias' in name:
      init.constant_(param, 0)

In [ ]:
import time

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
EPOCH = 200
#데이터셋의 크기가 작아 BATCH_SIZE는 10 이상이 적절할 것 같습니다. 실험을 통해 결정해주세요.
#모델이 깊지 않으므로 100 에포크 이상 학습할 필요가 없어 보입니다. 일단은 10 에포크에서 20 에포크 사이를 추천합니다.

model = MLP(8, device = DEVICE) #node 개수 또한 실험을 통해 결정해주세요.
model = model.to(DEVICE)
init_weights(model)
loss = nn.BCELoss()
optimizer =  torch.optim.SGD(model.parameters(), lr = 0.01) #lr은 학습률을 의미합니다. 0.1, 0.05, 0.01, 0.005, 0.001 총 5가지 학습률에 대해 실험을 진행해주시면 될 것 같습니다.

start_time = time.time()
train_loss_lst, test_loss_lst = [],[]
for epoch in range(EPOCH):
  model.train()

  for batch_idx, ( _kinds, _x, _target_scores) in enumerate(training_loader):
    kinds = torch.stack([torch.matmul(_kinds[i].to(torch.float32), model.weight_0.to(torch.float32)) for i in range(len(_kinds))])

    log_columns = [0, 2, 3]
    for col in log_columns:
        non_zero_indices = _x[:, col] != 0
        _x[non_zero_indices, col] = torch.log(_x[non_zero_indices, col])

    x = torch.cat((kinds.unsqueeze(1),_x), dim=1)
    x = model(x.to(DEVICE))
    target_scores = _target_scores/5
    cost = loss(Mapping(x), target_scores.unsqueeze(1))
    optimizer.zero_grad()

    cost.backward()
    optimizer.step()

    batch_idx += 1

    if (batch_idx % 5 == 0):
      print (f'Epoch: {epoch:03d}/{EPOCH:03d} | '
             f'Batch {batch_idx:03d}/{len(training_loader):03d} |'
             f' Cost: {cost:.4f}')

  model.eval()

  with torch.set_grad_enabled(False):
    train_loss = compute_accuracy_and_loss(model, training_loader, device=DEVICE)
    test_loss = compute_accuracy_and_loss(model, test_loader, device=DEVICE)
    train_loss_lst.append(train_loss)
    test_loss_lst.append(test_loss)

  print(f'Epoch: {epoch:03d}/{EPOCH:03d}')

  elapsed = (time.time() - start_time)/60
  print(f'Time elapsed: {elapsed:.2f} min')

  print(Mapping(x), target_scores.unsqueeze(1))

elapsed = (time.time() - start_time)/60
print(f'Total Training Time: {elapsed:.2f} min')

PATH = f"/content/drive/MyDrive/Toy_project_weight.pth"  #최종 모델의 가중치 저장
torch.save(model.state_dict(), PATH)

<ipython-input-108-1440b87b2d5e>:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  scaled_score= F.softmax(score)


Epoch: 000/200 | Batch 005/019 | Cost: 0.6888
Epoch: 000/200 | Batch 010/019 | Cost: 0.6848
Epoch: 000/200 | Batch 015/019 | Cost: 0.6842
Epoch: 000/200
Time elapsed: 0.01 min
tensor([[0.5200],
        [0.5200],
        [0.5200],
        [0.5200],
        [0.5200],
        [0.5200],
        [0.5200],
        [0.5200]], grad_fn=<UnsqueezeBackward0>) tensor([[0.6000],
        [0.7000],
        [0.8000],
        [0.6000],
        [0.9000],
        [0.4000],
        [0.8000],
        [0.6000]])
Epoch: 001/200 | Batch 005/019 | Cost: 0.6693
Epoch: 001/200 | Batch 010/019 | Cost: 0.6696
Epoch: 001/200 | Batch 015/019 | Cost: 0.6605
Epoch: 001/200
Time elapsed: 0.02 min
tensor([[0.5385],
        [0.5385],
        [0.5385],
        [0.5385],
        [0.5385],
        [0.5385],
        [0.5385],
        [0.5385]], grad_fn=<UnsqueezeBackward0>) tensor([[1.0000],
        [0.7000],
        [0.7000],
        [0.8000],
        [0.2000],
        [0.8000],
        [0.8000],
        [0.8000]])
Epoch: 0

In [ ]:
x = ['한식', math.log(1000), 4.33, math.log(190), math.log(158)]
x[0] = kind[x[0]]

In [ ]:
tensor = torch.tensor(x).unsqueeze(0)
tensor = model(tensor.to(DEVICE))

In [ ]:
print(Mapping(tensor))

## 7. coordinate function


> 주소를 입력하면 좌표를 불러오는 함수입니다.
오류가 나면 chatGPT와 https://api.ncloud-docs.com/docs/ai-naver-mapsgeocoding-geocode 사이트를 참고해주세요.

In [ ]:
import requests

def import_coordinate(id, secret, address):
  url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
  headers = {
    "X-NCP-APIGW-API-KEY-ID": id,
    "X-NCP-APIGW-API-KEY": secret,
  }

  params = {
    "query": address,
  }

  response = requests.get(url, params=params, headers=headers)
  if response.status_code == 200:
    data = response.json()

    coordinate_x = data.get('addresses', [])[0].get('x')
    coordinate_y = data.get('addresses', [])[0].get('y')

    return coordinate_x, coordinate_y
  else:
    raise RuntimeError()

## 8. duration function



> 입력받은 좌표를 토대로 최단 경로의 최단 시간을 리턴하는 함수입니다.오류가 나면 chatGPT와 https://api.ncloud-docs.com/docs/ai-naver-mapsdirections-driving 사이트를 참고해주세요.

In [ ]:
def return_duration(start_x, start_y, goal_x, goal_y):
  url = "https://naveropenapi.apigw.ntruss.com/map-direction-15/v1/driving"

  headers = {
    "X-NCP-APIGW-API-KEY-ID": str(id),
    "X-NCP-APIGW-API-KEY": str(secret)
  }

  params = {
    "start": "%f,%f" % (start_x, start_y),
    "goal": "%f,%f" % (goal_x, goal_y),
    "option": "trafast"  # 최단거리
  }

  response = requests.get(url, headers=headers, params=params)

  if response.status_code == 200:
    data = response.json()

    duration_milliseconds = data["route"]["trafast"][0]["summary"]["duration"]
    duration_seconds = duration_milliseconds / 1000

    return duration_seconds
  else:
    raise RuntimeError()

## 9. Test

In [ ]:
#아래 주소는 적절하게 바꾸면 됩니다.
#youngtong.csv 파일도 동일한 csv 파일 양식이라는 가정 하에 작성되었습니다.
#youngtong.csv 파일의 주관적 점수는 중요하지 않으니 대충 아무 숫자로 채워 넣으시면 됩니다.

youngtong_csv = '/content/drive/MyDrive/youngtong.csv'
youngtong_data = pd.read_csv(youngtong_csv, header=0)
X_youngtong, y_youngtong = youngtong_data.iloc[:, :6], youngtong_data.iloc[:, 6]
youngtong_dataset = CustomDataset(X_youngtong, y_youngtong)